# Tugas pengelasan teks

Seperti yang telah kami nyatakan, kami akan memberi tumpuan kepada tugas pengelasan teks yang mudah berdasarkan dataset **AG_NEWS**, iaitu untuk mengelaskan tajuk berita kepada salah satu daripada 4 kategori: Dunia, Sukan, Perniagaan dan Sains/Teknologi.

## Dataset

Dataset ini telah tersedia dalam modul [`torchtext`](https://github.com/pytorch/text), jadi kita boleh mengaksesnya dengan mudah.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Di sini, `train_dataset` dan `test_dataset` mengandungi koleksi yang mengembalikan pasangan label (nombor kelas) dan teks masing-masing, sebagai contoh:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Jadi, mari cetak 10 tajuk berita baru pertama daripada set data kita:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Oleh kerana set data adalah pengulang, jika kita ingin menggunakan data beberapa kali kita perlu menukarkannya kepada senarai:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Penjanaan Token

Sekarang kita perlu menukar teks kepada **nombor** yang boleh diwakili sebagai tensor. Jika kita mahukan perwakilan pada peringkat perkataan, kita perlu melakukan dua perkara:
* gunakan **penjana token** untuk memecahkan teks kepada **token**
* bina **perbendaharaan kata** daripada token-token tersebut.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Dengan menggunakan perbendaharaan kata, kita boleh dengan mudah mengekodkan rentetan token yang telah ditokenkan ke dalam satu set nombor:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Representasi Teks Bag of Words

Oleh kerana perkataan mewakili makna, kadangkala kita boleh memahami maksud sesuatu teks hanya dengan melihat perkataan individu, tanpa menghiraukan susunan mereka dalam ayat. Sebagai contoh, apabila mengklasifikasikan berita, perkataan seperti *cuaca*, *salji* mungkin menunjukkan *ramalan cuaca*, manakala perkataan seperti *saham*, *dolar* mungkin merujuk kepada *berita kewangan*.

**Bag of Words** (BoW) adalah representasi vektor yang paling biasa digunakan dalam kaedah tradisional. Setiap perkataan dikaitkan dengan indeks vektor, dan elemen vektor mengandungi bilangan kemunculan sesuatu perkataan dalam dokumen tertentu.

![Imej menunjukkan bagaimana representasi vektor bag of words diwakili dalam memori.](../../../../../translated_images/ms/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Anda juga boleh menganggap BoW sebagai jumlah semua vektor satu-hot-encoded untuk setiap perkataan individu dalam teks.

Di bawah adalah contoh bagaimana untuk menghasilkan representasi bag of words menggunakan pustaka python Scikit Learn:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Untuk mengira vektor bag-of-words daripada representasi vektor dataset AG_NEWS kita, kita boleh menggunakan fungsi berikut:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Nota:** Di sini kita menggunakan pembolehubah global `vocab_size` untuk menentukan saiz lalai perbendaharaan kata. Oleh kerana saiz perbendaharaan kata sering kali sangat besar, kita boleh mengehadkan saiz perbendaharaan kata kepada perkataan yang paling kerap digunakan. Cuba kurangkan nilai `vocab_size` dan jalankan kod di bawah, dan lihat bagaimana ia mempengaruhi ketepatan. Anda sepatutnya menjangkakan sedikit penurunan ketepatan, tetapi tidak secara dramatik, sebagai ganti prestasi yang lebih tinggi.


## Melatih Pengelas BoW

Sekarang setelah kita mempelajari cara membina representasi Bag-of-Words untuk teks kita, mari kita latih sebuah pengelas di atasnya. Pertama, kita perlu menukar dataset kita untuk latihan dengan cara di mana semua representasi vektor posisi ditukar kepada representasi bag-of-words. Ini boleh dicapai dengan menghantar fungsi `bowify` sebagai parameter `collate_fn` kepada `DataLoader` standard torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Sekarang mari kita definisikan rangkaian neural pengklasifikasi mudah yang mengandungi satu lapisan linear. Saiz vektor input adalah sama dengan `vocab_size`, dan saiz output sepadan dengan bilangan kelas (4). Oleh kerana kita sedang menyelesaikan tugas pengklasifikasian, fungsi pengaktifan akhir adalah `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Sekarang kita akan mendefinisikan gelung latihan standard PyTorch. Oleh kerana dataset kita agak besar, untuk tujuan pengajaran kita akan melatih hanya untuk satu epoch, dan kadang-kadang kurang daripada satu epoch (menentukan parameter `epoch_size` membolehkan kita menghadkan latihan). Kita juga akan melaporkan ketepatan latihan terkumpul semasa latihan; kekerapan pelaporan ditentukan menggunakan parameter `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams dan N-Grams

Satu kelemahan pendekatan bag of words ialah beberapa perkataan merupakan sebahagian daripada ungkapan berbilang perkataan, contohnya, perkataan 'hot dog' mempunyai makna yang sangat berbeza daripada perkataan 'hot' dan 'dog' dalam konteks lain. Jika kita sentiasa mewakili perkataan 'hot' dan 'dog' dengan vektor yang sama, ia boleh mengelirukan model kita.

Untuk mengatasi masalah ini, **representasi N-gram** sering digunakan dalam kaedah pengelasan dokumen, di mana kekerapan setiap perkataan, bi-perkataan atau tri-perkataan adalah ciri yang berguna untuk melatih pengelas. Dalam representasi bigram, sebagai contoh, kita akan menambah semua pasangan perkataan ke dalam perbendaharaan kata, selain daripada perkataan asal.

Di bawah adalah contoh bagaimana untuk menghasilkan representasi bigram bag of words menggunakan Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Kelemahan utama pendekatan N-gram ialah saiz perbendaharaan kata mula berkembang dengan sangat pantas. Dalam praktiknya, kita perlu menggabungkan representasi N-gram dengan beberapa teknik pengurangan dimensi, seperti *embeddings*, yang akan kita bincangkan dalam unit seterusnya.

Untuk menggunakan representasi N-gram dalam dataset **AG News** kita, kita perlu membina perbendaharaan kata ngram khas:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Kita boleh menggunakan kod yang sama seperti di atas untuk melatih pengklasifikasi, namun, ia akan menjadi sangat tidak cekap dari segi memori. Dalam unit seterusnya, kita akan melatih pengklasifikasi bigram menggunakan embeddings.

> **Nota:** Anda hanya boleh meninggalkan ngram yang muncul dalam teks lebih daripada jumlah yang ditentukan. Ini akan memastikan bigram yang jarang berlaku akan diabaikan, dan akan mengurangkan dimensi dengan ketara. Untuk melakukan ini, tetapkan parameter `min_freq` kepada nilai yang lebih tinggi, dan perhatikan perubahan panjang perbendaharaan kata.


## Kekerapan Istilah dan Frekuensi Dokumen Songsang TF-IDF

Dalam representasi BoW, kemunculan perkataan diberi berat yang sama, tanpa mengira perkataan itu sendiri. Walau bagaimanapun, jelas bahawa perkataan yang kerap muncul, seperti *a*, *in*, dan sebagainya, adalah jauh kurang penting untuk pengelasan berbanding istilah khusus. Malah, dalam kebanyakan tugas NLP, sesetengah perkataan lebih relevan daripada yang lain.

**TF-IDF** adalah singkatan kepada **kekerapan istilah–frekuensi dokumen songsang**. Ia adalah variasi daripada bag of words, di mana nilai binari 0/1 yang menunjukkan kemunculan perkataan dalam dokumen digantikan dengan nilai titik terapung, yang berkaitan dengan kekerapan kemunculan perkataan dalam korpus.

Secara lebih formal, berat $w_{ij}$ bagi perkataan $i$ dalam dokumen $j$ ditakrifkan sebagai:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
di mana
* $tf_{ij}$ adalah bilangan kemunculan $i$ dalam $j$, iaitu nilai BoW yang telah kita lihat sebelum ini
* $N$ adalah bilangan dokumen dalam koleksi
* $df_i$ adalah bilangan dokumen yang mengandungi perkataan $i$ dalam keseluruhan koleksi

Nilai TF-IDF $w_{ij}$ meningkat secara berkadar dengan bilangan kali perkataan muncul dalam dokumen dan diselaraskan dengan bilangan dokumen dalam korpus yang mengandungi perkataan tersebut, yang membantu menyesuaikan fakta bahawa sesetengah perkataan muncul lebih kerap daripada yang lain. Sebagai contoh, jika perkataan muncul dalam *setiap* dokumen dalam koleksi, $df_i=N$, dan $w_{ij}=0$, dan istilah tersebut akan diabaikan sepenuhnya.

Anda boleh dengan mudah mencipta vektorisasi TF-IDF teks menggunakan Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Kesimpulan

Walaupun representasi TF-IDF memberikan berat kekerapan kepada pelbagai perkataan, ia tidak mampu mewakili makna atau susunan. Seperti yang dikatakan oleh ahli bahasa terkenal J. R. Firth pada tahun 1935, “Makna lengkap sesuatu perkataan sentiasa bersifat kontekstual, dan tiada kajian tentang makna yang terpisah daripada konteks boleh dianggap serius.”. Kita akan belajar kemudian dalam kursus ini bagaimana untuk menangkap maklumat kontekstual daripada teks menggunakan pemodelan bahasa.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat penting, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
